In [1]:
!pip install google-generativeai
!pip install google-cloud-aiplatform

In [2]:
import google.cloud.aiplatform as aiplatform

In [3]:
import google.generativeai as genai
from google.colab import userdata

gemini_key = userdata.get('gemini_key')

genai.configure(api_key=gemini_key)

In [4]:
# configuration du modèle
generation_config = {
  "temperature": 0.9,
  "top_p": 1,
  "top_k": 5,
  "max_output_tokens": 2048,
}

In [5]:
# appel au modèle generative avec la configuration déjà fixée
model = genai.GenerativeModel(model_name= "gemini-1.0-pro",
                              generation_config = generation_config)


In [ ]:
import pandas as pd
df=pd.read_csv("/content/output_questions (2).tsv",sep="\t")
df.head()

,Question,Option A,Option B,Option C,Option D,Correct Answer,Explanation
0,Which MITRE ATT&CK technique involves an adver...,Spearphishing Attachment (T1193),Social Engineering (T1140),Remote Access Tools (T1219),Man-in-the-Middle Attack (T1557),B,Social Engineering (T1140) is the correct answ...
1,Which MITRE ATT&CK technique involves an adver...,(A) Exploit Public-Facing Application (T1190),(B) Third-party Software (T1072),(C) Exploitation for Client Execution (T1203),(D) Exploitation for Privilege Escalation (T1068),B,Adversaries can exploit weaknesses in third-pa...
2,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Data Staged (T1074),Command and Control (T1009),Remote File Copy (T1105),C,Command and Control (T1009) corresponds to adv...
3,"In the context of MITRE ATT&CK for Enterprise,...",T1548.002 - Abuse Elevation Control Mechanism,T1059.001 - Command-Line Interface,T1566.001 - Phishing,T1078.002 - Valid Accounts,T1548.002 - Abuse Elevation Control Mechanism,Adversaries may abuse elevation control mechan...
4,Which MITRE ATT&CK technique involves an adver...,Process Injection (T1055) - Process Injection,Man-in-the-Middle (T1557) - Man-in-the-Middle,Control Device Identification (T818) - Control...,Manipulate Device Communication (T848) - Manip...,C,Manipulate Device Communication (T848) involve...


In [ ]:
def get_model_response(question, options):
    prompt = (
        f"You are given a multiple-choice question (MCQ) from a Cyber Threat Intelligence (CTI) knowledge benchmark dataset.\n"
        f"Your task is to choose the best option among the four provided.\n\n"
        f"Question: {question}\n"
        f"A. {options[0]}\n"
        f"B. {options[1]}\n"
        f"C. {options[2]}\n"
        f"D. {options[3]}\n\n"
        f"Return your answer as a single uppercase letter: A, B, C, or D."
    )
    max_retries = 5
    backoff_time = 10  # initial backoff time in seconds

    for attempt in range(max_retries):
        try:
            # Generate the response
            response = model.generate_content(prompt)  # Replace with actual API call

            # Log the full response for debugging
            print(f"Response object: {response}")

            # Check if response.parts exists and is not empty
            if not hasattr(response, 'parts') or not response.parts:
                raise ValueError("No valid response parts found.")

            # Extract the text from the response object
            text = response.parts[0].text
            answer = text.strip()[-1].upper()  # Apply strip() to the extracted text
            return answer
        except TooManyRequests:
            if attempt < max_retries - 1:
                sleep_time = backoff_time * (2 ** attempt)  # Exponential backoff
                print(f"Rate limit hit. Retrying in {sleep_time} seconds...")
                time.sleep(sleep_time)
            else:
                raise
        except Exception as e:
            print(f"An error occurred: {e}")
            raise

In [ ]:
# Iterate over the sampled DataFrame rows
for index, row in sampled_df.iterrows():
    question = row['Question']
    options = [
        row['Option A'],
        row['Option B'],
        row['Option C'],
        row['Option D']
    ]

    # Get the model response for the current question
    try:
        response_text = get_model_response(question, options)
        print(f"Question {index + 1}: {response_text}")

        # Append the question and response to the output_data list
        output_data.append({
            'Question': question,
            'Option A': options[0],
            'Option B': options[1],
            'Option C': options[2],
            'Option D': options[3],
            'Geminai': response_text
        })
    except Exception as e:
        print(f"Failed to get a response for question {index + 1}: {e}")
        continue

# Create a new DataFrame from the output_data
output_df = pd.DataFrame(output_data)

# Save the new DataFrame to a CSV file
output_df.to_csv('geminai.csv', index=False)

Response object: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "D"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
      "usage_metadata": {
        "prompt_to

Rate limit hit. Retrying in 10 seconds...
Response object: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "D"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
  

Rate limit hit. Retrying in 10 seconds...


Rate limit hit. Retrying in 20 seconds...
Response object: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "C"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
  

Rate limit hit. Retrying in 10 seconds...


...

Rate limit hit. Retrying in 20 seconds...


Rate limit hit. Retrying in 40 seconds...
Response object: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "D"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
  

Rate limit hit. Retrying in 10 seconds...


Rate limit hit. Retrying in 20 seconds...


Rate limit hit. Retrying in 40 seconds...
Response object: response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "B"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0,
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            }
          ]
        }
      ],
  

In [ ]:
answer=pd.read_csv("/content/geminai.csv")
answer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133 entries, 0 to 132
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Question  133 non-null    object
 1   Option A  133 non-null    object
 2   Option B  133 non-null    object
 3   Option C  133 non-null    object
 4   Option D  133 non-null    object
 5   Geminai   133 non-null    object
dtypes: object(6)
memory usage: 6.4+ KB


In [ ]:
df_merged = pd.merge(df, answer, on='Question', how='inner')
df_merged.head()

,Question,Option A_x,Option B_x,Option C_x,Option D_x,Correct Answer,Explanation,Option A_y,Option B_y,Option C_y,Option D_y,Geminai
0,Which MITRE ATT&CK technique involves an adver...,Exfiltration Over Alternative Protocol (T1048),Remote Desktop Protocol (T1076),Packet Sniffing (T1040),Exploitation for Defense Evasion (T1211),C,Packet Sniffing (T1040) is the correct answer ...,Exfiltration Over Alternative Protocol (T1048),Remote Desktop Protocol (T1076),Packet Sniffing (T1040),Exploitation for Defense Evasion (T1211),C
1,"In the context of MITRE ATT&CK Enterprise, whi...",Account Discovery (T1087),External Remote Services (T1133),Valid Accounts (T1078),Inbox Rules (T1114),D,Inbox Rules (T1114) allows an adversary to man...,Account Discovery (T1087),External Remote Services (T1133),Valid Accounts (T1078),Inbox Rules (T1114),D
2,"In the context of MITRE ATT&CK Enterprise, whi...",Account Discovery (T1087),External Remote Services (T1133),Valid Accounts (T1078),Inbox Rules (T1114),D,Inbox Rules (T1114) allows an adversary to man...,Account Discovery (T1087),External Remote Services (T1133),Valid Accounts (T1078),Inbox Rules (T1114),D
3,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Execution through API (T1106),Trusted Relationship (T1199),Trap (T1154),A,Process Injection (T1055) refers to an attacke...,Process Injection (T1055),Execution through API (T1106),Trusted Relationship (T1199),Trap (T1154),C
4,"In the context of MITRE ATT&CK for Enterprise,...",Process Injection (T1055),Execution through API (T1106),Trusted Relationship (T1199),Trap (T1154),A,Process Injection (T1055) refers to an attacke...,Process Injection (T1055),Execution through API (T1106),Trusted Relationship (T1199),Trap (T1154),A


In [ ]:
def compute_mcq_accuracy(df):
    correct = 0
    total = 0
    for idx, row in df.iterrows():
        pred = row['Geminai']
        gt = row['Correct Answer']
        if pred in ['A', 'B', 'C', 'D', 'X']:
            total += 1
        else:
            print('Invalid response at row {}'.format(idx+1))
        if pred == gt:
            correct += 1
    return correct/total*100

In [ ]:
print('Accuracy:', compute_mcq_accuracy(df_merged))

Accuracy: 63.57142857142857


## Génération avec flux
Par défaut, le modèle renvoie une réponse à la fin du processus de génération. Vous pouvez également diffuser la réponse en flux continu au fur et à mesure de sa génération. Le modèle renvoie alors des fragments de la réponse dès qu'ils sont générés.